# Analysis of an unbalanced 3-phase LV network

### Introduction

This tutorial is meant to introduce the procedure for modelling components of a given low voltage (LV) network using the *Roseau Load Flow* solver. It will also cover the process of running a power flow, accessing results, and analysis such as voltage regulation and energy losses.

Consider the simple LV network shown in the figure below containing a MV/LV, $\Delta$-Y transformer (11kV/0.4kV, 250 kVA) between the source bus and bus A, a 3-phase line connecting buses A and B, and three single-phase lines connecting bus B with buses C, D and E each of which serves as a connection point for a house. 

<center> <img style="float: middle;" 
          src="./data/Simple_LV_Network.png" 
          alt="Simple LV network"
          width="40%"> 
</center>

**<center> Figure 1. Simple LV Network</center>**

### Import the required modules and functions 
First we need to import the modules and functions needed for this tutorial. The purpose of the following code are explained as follows: 

* `from roseau.load_flow import *`: imports all modules, classes and functions in the `load_flow` submodule of the *Roseau Load Flow* package. These classes are needed to model the various components of the network such as buses, lines, transformers, etc. 

* `import numpy as np`: imports the *numpy* package which will be used for manipulating parameter arrays, calculating exponents, and for retrieving the pi constant.

In [ ]:
import numpy as np
from roseau.load_flow import *
from roseau.load_flow.converters import calculate_voltages

#### Creating the buses

After importing all the necessary modules, we can move on to the modelling of the given LV network. We start with the buses which are the nodes of the network. To create a bus in *Roseau Load Flow* (RLF), we use the [`Bus`](https://roseau-load-flow.roseautechnologies.com/models/Bus.html) class and pass the constructor an identifier and the number of phases that the bus has.

The identifier is used internally by the solver to identify network elements and is also used to name the element when displaying results. A bus in *RLF* can also have either be a three-phase network (with or without neutral i.e. `abcn` or `abc`) or any combination of one/two phases with or without the neutral. More information on buses and possible phase combinations in *RLF* can be found [here](https://roseau-load-flow.roseautechnologies.com/models/Bus.html). 

For this tutorial, the network's buses are created as follows:

* `source_bus = Bus(id='source_bus', phases='abc')`: this creates a 3-phase bus with no neutral since it's typically not needed in a transmission network.

* `bus_a = Bus(id="bus_A", phases="abcn")`: this creates a 3-phase bus with a neutral since it's a distribution network bus. Bus B is created in a similar manner.

* `bus_c = Bus(id="bus_C", phases="an")`: this creates a single phase bus with a neutral. Buses D & E are created in a similar manner but with different phases. 

In [ ]:
source_bus = Bus(id="source_bus", phases="abc")
bus_a = Bus(id="bus_A", phases="abcn")
bus_b = Bus(id="bus_B", phases="abcn")
bus_c = Bus(id="bus_C", phases="an")
bus_d = Bus(id="bus_D", phases="bn")
bus_e = Bus(id="bus_E", phases="cn")

#### Adding a voltage source to the source bus

Because the source bus is the driving force of this network, we need to add a voltage source at the bus to reflect this. To add a voltage source to a bus in *RLF*, we first need to set its voltage value. Since the source bus is a 3-phase bus, we need a set of three-phase voltages (one for each phase). 

A voltage source can either be $\Delta$ or Y-connected as described [here](https://roseau-load-flow.roseautechnologies.com/models/VoltageSource.html). Since the source bus has no neutral, therefore, we can only add a $\Delta$-connected voltage source to it. This also means we need to do transform the phase voltages from the Y-connected source bus to match that of the voltage source. 

<center> <img style="float: middle;" 
          src="./data/Star_Delta.png" 
          alt="Star-Delta voltage transformation"
          width="40%"> 
</center>

**<center> Figure 2. Transforming phase voltages between Y & $\Delta$ connections</center>**

The complete procedure is as follows:
* `v_star = (20e3/np.sqrt(3)) * np.exp([0, -2j * np.pi / 3, 2j * np.pi / 3]))`: creates a complex array of three-phase voltages with a magnitude of $\frac{20}{\sqrt{3}}$ kV and a phase shift of 120 degrees between them.

* `v_delta = calculate_voltages(v_star)`: converts the phase voltages of the source bus to those of the voltage source using the `calculate_voltages` function imported at the start of the tutorial

Next, we create the voltage source itself using the [`VoltageSource`](https://roseau-load-flow.roseautechnologies.com/models/VoltageSource.html) class and specify an id, the bus it is connected to and the calculated voltages as described below:

* `vs = VoltageSource(id="vs", bus=source_bus, voltages=v_delta)`

In [ ]:
v_star = (20e3 / np.sqrt(3)) * np.exp([0, -2j * np.pi / 3, 2j * np.pi / 3])
v_delta = calculate_voltages(v_star, phases="abc")
vs = VoltageSource(id="vs", bus=source_bus, voltages=v_delta)

#### Adding the transformer

Next, we will add the MV/LV transformer. To add a transformer in *RLF*, first we need to define the transformer's parameters using the [`TransformerParameters`](https://roseau-load-flow.roseautechnologies.com/models/Transformer/index.html) class. This can be done in two ways: manually or using the catalogue. Manual entry involves inputting seven transformer parameters as described [here](https://roseau-load-flow.roseautechnologies.com/models/Transformer/index.html) 

This tutorial uses the second method which involves using predefined transformer parameters from the catalogue of transformers included in *RLF*. In this case, we use the parameters of a 250 kVA Schneider Electric Minera transformer with the highest efficiency (AA0Ak) as shown below. More information on retrieving parameters from the catalogue can be seen [here](https://roseau-load-flow.roseautechnologies.com/usage/Catalogues.html)


In [ ]:
tp = TransformerParameters.from_catalogue(id="SE_Minera_AA0Ak_250kVA")

Once the transformer's parameters have been defined, a transformer can then be created using the [`Transformer`](https://roseau-load-flow.roseautechnologies.com/models/Transformer/index.html) class. We pass in the id of the transformer as well as the buses & phases on its primary side (1) and secondary side (2). Finally, we pass in the transformer's parameters created above. 


In [ ]:
transformer = Transformer(id="transf", bus1=source_bus, bus2=bus_a, phases1="abc", phases2="abcn", parameters=tp)

#### Adding potential references and grounds

To run power flow calculations in *RLF*, a potential reference must be added to an element in each galvanically isolated section of the network. For this tutorial, the isolated sections are the primary and secondary sides of the transformer.

For the primary side of the transformer, we use a [`PotentialRef`](https://roseau-load-flow.roseautechnologies.com/models/PotentialRef.html) connected to the `source_bus`.

In [ ]:
pref_mv = PotentialRef(id="pref_mv", element=source_bus)

The constructor of the `PotentialRef` class takes an `id` as all elements in *RLF*, an `element` argument to select the element (`Bus` or `Ground`) to set the potential to 0 V and an optional `phase` argument which is set to `None` by default. In this case, as the bus `source_bus` has three potentials (`a`, `b` and `c`) and as the `phase` argument is not defined, the sum of these potentials is set to 0 V by `pref_mv`. 


For the LV part of the network, lines with shunt will be defined. They will require a [`Ground`](https://roseau-load-flow.roseautechnologies.com/models/Ground.html) element. The constructor of this element only takes an id.

In [ ]:
ground = Ground(id="ground")

Now, we can connect this ground to the neutral of the secondary side of the transformer using its `connect` method. 

In [ ]:
ground.connect(bus=bus_a, phase="n")

For now, the potential of the element `ground` has not been set to 0 V. In *RLF*, the ability to define `Ground` elements has been separated from the ability to set potentials to zero. It may be hard to accept in this first tutorial, but it may become more clear with the following tutorials.

In order to set the potential of the element `ground` to 0 V, we just create another [`PotentialRef`](https://roseau-load-flow.roseautechnologies.com/models/PotentialRef.html).


In [ ]:
pref_lv = PotentialRef(id="pref_lv", element=ground)

Here, the element provided to the constructor is a `Ground` element. As a `Ground` has a single potential, the `phase` argument of the constructor must be ignored.

#### Connecting buses with lines

Next, we'll add in all the lines present in the network. Similar to the transformer, to add lines in *RLF*, we first need to specify the parameters for the line and this can be done manually using the [`LineParameters`](https://roseau-load-flow.roseautechnologies.com/models/Line/Parameters.html) or with the catalogue. For manual entry, we will need to specify the series impedance matrix (`z_line`) and optionally, the shunt admittance matrix (`y_shunt`). The dimensions of these matrices must be equal to the number of conductors.

Just as we did previously, we will use the *RLF* catalogue to retrieve predefined parameters for lines. For the three-phase line between buses A & B, we will use parameters from an overhead aluminium cable with a cross-sectional area of 240 mm². The single phase lines will be modelled using parameters from an underground aluminium cable with a cross-sectional area of 19 mm² as shown below. Line parameters make it easy to create multiple lines with the same characteristics. 
* `lp_240 = LineParameters.from_catalogue(name='O_AL_240')`  
* `lp_19 = LineParameters.from_catalogue(name='U_AL_19')`

However, all the line parameters in the catalogue are 3-phase lines with no neutrals. This means  the size of the matrices are 3x3. Since we have a neutral in our three-phase line (4x4) as well as single phase lines (2x2), then we need to resize the default matrices. Assuming all conductors are the same, we only need to retrieve the impedance and admittance of one conductor as shown below.
* `z_240, y_240 = (lp_240.z_line.m[0,0], lp_240.y_shunt.m[0,0])`

Next, we can replicate these values to create new `z_line` and `y_shunt` matrices with the correct dimensions. And then, we create a new line parameter object with these new values while keeping the same maximum current value as the old one. 
* `z_240, y_240 = [np.eye(4, dtype=complex) * z_240, np.eye(4, dtype=complex) * y_240]`
* `new_lp_240 = LineParameters(id='new_lp_240', z_line=z_240, y_shunt=y_240, max_current=lp_240.max_current)`


Finally, we create the actual lines using the [`Line`](https://roseau-load-flow.roseautechnologies.com/models/Line/index.html) class. To do this, we pass the constructor the identifier, buses, length, phase, and parameters of the lines as shown below. 
* `line_ab = Line(id="lineA_B", bus1=bus_a, bus2=bus_b, parameters=new_lp_240, phases='abcn', length=Q_(1.0, "km"), ground=ground)`

**Note**:  In *RLF*, when the `y_shunt` matrix is given in the line parameters, we must define a ground to be used for shunt connections. Here, we reuse the same ground for the LV part of the network.

In [ ]:
# Getting line parameters from the catalogue
lp_240 = LineParameters.from_catalogue(name="O_AL_240")
lp_19 = LineParameters.from_catalogue(name="U_AL_19")

# TODO Replace this section by a catalogue with lines with neutral
# Getting the z and y values for both line sections
z_240, y_240 = (lp_240.z_line.m[0, 0], lp_240.y_shunt.m[0, 0])
z_19, y_19 = (lp_19.z_line.m[0, 0], lp_19.y_shunt.m[0, 0])

# Resizing the matrices
z_240, y_240 = [np.eye(4, dtype=complex) * z_240, np.eye(4, dtype=complex) * y_240]
z_19, y_19 = [np.eye(2, dtype=complex) * z_19, np.eye(2, dtype=complex) * y_19]

# Make new line parameters
new_lp_240 = LineParameters(id="new_lp_240", z_line=z_240, y_shunt=y_240, max_current=lp_240.max_current)
new_lp_19 = LineParameters(id="new_lp_19", z_line=z_19, y_shunt=y_19, max_current=lp_19.max_current)

# Creating the actual lines
line_ab = Line(
    id="lineA_B", bus1=bus_a, bus2=bus_b, parameters=new_lp_240, phases="abcn", length=Q_(1.0, "km"), ground=ground
)
line_bc = Line(
    id="lineB_C", bus1=bus_b, bus2=bus_c, phases="an", parameters=new_lp_19, length=Q_(10, "m"), ground=ground
)
line_bd = Line(
    id="lineB_D", bus1=bus_b, bus2=bus_d, phases="bn", parameters=new_lp_19, length=Q_(10, "m"), ground=ground
)
line_be = Line(
    id="lineB_E", bus1=bus_b, bus2=bus_e, phases="cn", parameters=new_lp_19, length=Q_(10, "m"), ground=ground
)

Please note that as for most arguments in *RLF* functions, methods and classes, the arguments can be given with a unit using the `Q_` (**Q** for **Q**uantities) constructor. Here by instance, the length of the lines were given in kilometers (for the first line) while the others were given in meters.

#### Adding the loads

In the final modelling step, we will add the loads at buses C, D & E. The description of the loads are given in the table below. *RLF* can model different types of loads as described [here](https://roseau-load-flow.roseautechnologies.com/models/Load/index.html). 

| Load Name | Phases | Connected bus | Peak Demand (kW) | PF   |
|:----------|:-------|:--------------|:-----------------|:-----|
| Load_1    | 1      | C             | 7                | 0.95 |
| Load_2    | 1      | D             | 6                | 0.95 |
| Load_3    | 1      | E             | 8                | 0.95 |

For this tutorial, the loads are assumed to be drawing constant power irrespective of the voltage. This means we can model them using a [`PowerLoad`](https://roseau-load-flow.roseautechnologies.com/models/Load/PowerLoad.html) class. This class takes in the id of the load, the bus and bus phase it is attached to and the apparent power of the load in complex number form. The powers must be a set of one or three values for single & 3-phase loads respectively.

Since we are given active power instead of apparent power, then we need to carry out a conversion at the given power factor. For this we have written a small function, `conv_func` that takes in the peak demand and the power factor to carry out the calculation below. 

$$\underline{S} = P \left[1 + j\tan\left(\cos^{-1}(pf)\right)\right]$$


In [ ]:
def conv_func(power: float, pf: float) -> complex:
    return power * (1 + np.tan(np.arccos(pf)) * 1j)

Then, we can finally create the load as follows:

In [ ]:
load1 = PowerLoad(id="load1", bus=bus_c, phases="an", powers=[conv_func(7e3, 0.95)])
load2 = PowerLoad(id="load2", bus=bus_d, phases="bn", powers=[conv_func(6e3, 0.95)])
load3 = PowerLoad(id="load3", bus=bus_e, phases="cn", powers=[conv_func(8e3, 0.95)])

Please note that these three loads are connected between a phase and the neutral of their bus which is defined using the same phases. It has two consequences:
* Précising the phases in the `PowerLoad` constructor is useless as the phases of the buses are taken by default.
* Moreover, the `powers` argument must be an array of length 1.  

#### Building the network

Following the modelling of all network components, the next step is to build the actual network. Currently, components only exist in isolation and they do not belong to any network. This can be verified by running the code below which returns nothing.


In [ ]:
load1.network is None

To build the network in *RLF*, we use the `ElectricalNetwork` class. This class provides several methods for constructing the network and perhaps the easiest one is the `from_element` method. This method allows you to create the entire network from a single bus and adds all the other elements automatically. So, we pass in the source bus to create the entire network as shown below.

In [ ]:
en = ElectricalNetwork.from_element(initial_bus=source_bus)

To verify if the network has been successfully created, we can just print `en` to see the number of elements in the built network.

In [ ]:
en

### Running a load flow

To execute load flow calculations in *RLF*, a license is required. However, a free, limited license key is available [`here`](https://roseau-load-flow.roseautechnologies.com/License.html#license-types) and can be used for the purpose of this tutorial. 

Once the license is properly installed, we can simply run a load flow as shown below:

In [ ]:
en.solve_load_flow()

This function returns the number of iterations performed by the solver and the residual error after convergence. 

#### Accessing results

After running the load flow calculations, we can extract the value of several quantities from different network elements. Each quantity can be accessed through the name of the quantity prefixed with the phrase `res_`. A complete breakdown of possible results for each network element is given below. 



| Element type                                | Available results                                                                                                                                       |
|---------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------|
| `Bus`                                       | `res_potentials`, `res_voltages`, `res_violated`                                                                                                        |
| `Line`                                      | `res_currents`, `res_powers`, `res_potentials`, `res_voltages`, `res_series_power_losses`, `res_shunt_power_losses`, `res_power_losses`, `res_violated` |
| `Transformer`                               | `res_currents`, `res_powers`, `res_potentials`, `res_voltages`, `res_violated`                                                                          |
| `Switch`                                    | `res_currents`, `res_powers`, `res_potentials`, `res_voltages`                                                                                          |
| `ImpedanceLoad`, `CurrentLoad`, `PowerLoad` | `res_currents`, `res_powers`, `res_potentials`, `res_voltages`, `res_flexible_powers`&#8270;                                                            |
| `VoltageSource`                             | `res_currents`, `res_powers`, `res_potentials`, `res_voltages`                                                                                          |
| `Ground`                                    | `res_potential`                                                                                                                                         |
| `PotentialRef`                              | `res_current` _(Always zero for a successful load flow)_                                                                                                |

&#8270;: `res_flexible_powers` is only available for flexible loads (`PowerLoad`s with `flexible_params`).


To access results for different elements in *RLF*, there are two main methods. The first method involves calling the `res_` method for the desired quantity for that element e.g.


In [ ]:
load1.res_voltages

As you can see, the results is provided as a quantity i.e. with a unit.

However, this can only be used for one element at a time. To extract results for multiple similar elements, we call the `res_` method of the electrical network instead e.g.

In [ ]:
en.res_loads

We get a data frame (from the `pandas` library) of results. In data frame, the units are not included thus the values are always retrieved in V, A and W.

##### Accessing active and reactive power of loads

Although, these were explicitly specified, we can still verify the results. We will apply both methods described above to retrieve the active (P) and reactive (Q) powers of the three loads present in the network.

In [ ]:
# Note: We need an accessor for the power dissipated by a PowerLoad in RLF
# Method 1
load_power = load1.res_voltages * np.conj(load1.res_currents[0])
load_power.to("kVA")

Here, the result is converted in kVA using the `to` method of quantities.

In [ ]:
# Method 2
# We extract the numpy arrays of results
load_volts = en.res_loads_voltages["voltage"].sort_index().values
load_currents = en.res_loads.loc[(slice(None), ("a", "b", "c")), "current"].sort_index().values

# We compute the results
load_powers = Q_(load_volts * np.conj(load_currents), "VA")
load_powers.to("kVA")

Here the results were converted in a quantity and displayed in kVA.

##### Voltage magnitude of the load and 3-phase buses

Similar to the loads, we can extract the voltage magnitude of a certain bus using its `res_voltages` property. This returns the voltage in rectangular format which can be converted into polar form to extract the magnitude and phase angle.

To extract the results for multiple or all buses simultaneously, we can use the `res_buses` property of the electrical network. This returns a dataframe which can be filtered to get the results for the desired buses. By default, the results are also in rectangular format. To convert to polar form, we use the transform method of dataframes and apply two functions to get the magnitude and phase angle (in degrees) of the voltage.

Both methods are shown below. 

In [ ]:
# Method 1
np.abs(bus_c.res_voltages)

In [ ]:
# Because of a limitation of the `pint` library, the `angle` method of numpy is not well-supported.
Q_(np.angle(bus_c.res_voltages.m, deg=True), "degree")

In [ ]:
# Method 2
import functools as ft

en.res_buses_voltages["voltage"].transform([np.abs, ft.partial(np.angle, deg=True)])

#### Active and Reactive powers of the transformer

Similarly, to extract the active & reactive powers for the transformer, we can use the two methods of accessing results as shown below. It can be observed that the powers in the secondary side of the transformer are negative. This indicates that the power is leaving the transformer as opposed to the positive powers on the primary side entering the transformer. Also, there is no power flowing thorough the neutral phase as expected. 

In [ ]:
# Method 1
# Primary side powers (in kVA, rounded)
np.round(transformer.res_powers[0].to("kVA"), 3)

In [ ]:
# Secondary side powers (in kVA, rounded)
np.round(transformer.res_powers[1].to("kVA"), 3)

In [ ]:
# Method 2 (in kVA and rounded)
np.round(en.res_transformers[["power1", "power2"]] * 1e-3, 3)

#### Line Losses

Finally, the line losses on the network can also be conveniently accessed using the described methods. The first method uses the `res_series_power_losses` of each line and returns the line losses for each phase. The second method uses the `res_lines` property of the electrical network and the column `series_losses` in the resulting dataframe contains the power losses on each phase. 

In [ ]:
# Method 1
np.abs(line_ab.res_series_power_losses)

In [ ]:
# Method 2 (in W)
en.res_lines["series_losses"].transform([np.abs, ft.partial(np.angle, deg=True)])